# Applying CRISP-DM on the MIMIC-3 Dataset (Part II)
In this notebook, I apply the 3 last phases of the CRISP-DM process:

4. Modeling
5. Evaluation
6. Deployment

<img src="./media/crisp.png" width="30%">


A quick note here is that we prepare the data for training the model (it would be still related to the step 3). However, the main part of this notebook is related to the final 3 steps.

In [1]:
# !pip install awswrangler

In [2]:
#### Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.model_selection import cross_val_score, train_test_split

%matplotlib inline

#### The small part related to step 3. Data Preparation

In [3]:
database = 'mimiciii'

In [4]:
query = """
with ce as
(
  select
    icustay_id, charttime, itemid, valuenum
  from chartevents
  -- specify what data we want from chartevents
  where itemid in
  (
  211, -- Heart Rate
  618, --	Respiratory Rate
  615 --	Resp Rate (Total)
  )
  -- how did we know heart rates were stored using ITEMID 211? Simple, we looked in D_ITEMS!
  -- Try it for yourself: select * from d_items where lower(label) like '%heart rate%'
)
select
  -- ICUSTAY_ID identifies each unique patient ICU stay
  -- note that if the same person stays in the ICU more than once, each stay would have a *different* ICUSTAY_ID
  -- however, since it's the same person, all those stays would have the same SUBJECT_ID
  ie.icustay_id

  -- this is the outcome of interest: in-hospital mortality
  , max(adm.HOSPITAL_EXPIRE_FLAG) as OUTCOME

  -- this is a case statement - essentially an "if, else" clause
  , min(
      case
        -- if the itemid is 211
        when itemid = 211
          -- then return the actual value stored in VALUENUM
          then valuenum
        -- otherwise, return 'null', which is SQL standard for an empty value
        else null
      -- end the case statement
      end
    ) as HeartRate_Min

    -- note we wrapped the above in "min()"
    -- this takes the minimum of all values inside, and *ignores* nulls
    -- by calling this on our case statement, we are ignoring all values except those with ITEMID = 211
    -- since ITEMID 211 are heart rates, we take the minimum of only heart rates

  , max(case when itemid = 211 then valuenum else null end) as HeartRate_Max
  , min(case when itemid in (615,618) then valuenum else null end) as RespRate_Min
  , max(case when itemid in (615,618) then valuenum else null end) as RespRate_Max
from icustays ie

-- join to the admissions table to get hospital outcome
inner join admissions adm
  on ie.hadm_id = adm.hadm_id

-- join to the chartevents table to get the observations
left join ce
  -- match the tables on the patient identifier
  on ie.icustay_id = ce.icustay_id
  -- and require that the observation be made after the patient is admitted to the ICU
  and ce.charttime >= ie.intime
  -- and *before* their admission time + 1 day, i.e. the observation must be made on their first day in the ICU
  and ce.charttime <= ie.intime + interval '1' day
group by ie.icustay_id
order by ie.icustay_id
"""

data = wr.athena.read_sql_query(query,database)
data.drop('icustay_id', axis=1, inplace=True)

In [5]:
data.head()

,outcome,heartrate_min,heartrate_max,resprate_min,resprate_max
0,0,NaN,NaN,NaN,NaN
1,0,72.0,122.0,14.0,39.0
2,0,62.0,84.0,14.0,27.0
3,0,80.0,104.0,13.0,29.0
4,0,88.0,106.0,13.0,22.0


In [6]:
# We should have +60K rows in our dataset represeting patients from the MIMIC-III dataset
data.shape

(61532, 5)

## 4. Modeling

Here we develop, train and test the model locally.

In [7]:
# move from a data frame into a numpy array
X = data.iloc[:,1:].to_numpy()
y = data['outcome'].to_numpy()

In [8]:
# Cast to numeric types
X = X.astype(float)
y = y.astype(int)

In [9]:
# split data in 80%-20% training/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
# impute mean for missing values, since we have only numerical values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
model = LogisticRegression(fit_intercept=True, solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## 5. Evaluation

We evaluate our model locally, analyzing the test dataset and using AUROC, confusion matrices and accuracy:

In [12]:
# predict class labels for the test set
y_pred = model.predict(X_test)

# generate class probabilities
y_prob = model.predict_proba(X_test)

# generate evaluation metrics
print('Accuracy = {}'.format(accuracy_score(y_test, y_pred)))
print('AUROC = {}'.format(roc_auc_score(y_test, y_prob[:, 1])))

print('\nConfusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\nClassification report')
print(classification_report(y_test, y_pred))

Accuracy = 0.8951816039652231
AUROC = 0.6033460168207213

Confusion matrix
[[10964    15]
 [ 1275    53]]

Classification report
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     10979
           1       0.78      0.04      0.08      1328

    accuracy                           0.90     12307
   macro avg       0.84      0.52      0.51     12307
weighted avg       0.88      0.90      0.85     12307



In [13]:
# evaluate a logistic regression with 5-fold cross-validation
estimator = Pipeline([("imputer", SimpleImputer(missing_values=np.nan,
                                          strategy="mean")),
                      ('scaler', StandardScaler()),
                      ("regression", LogisticRegressionCV(cv=5,
                                                          scoring='roc_auc',
                                                          solver='lbfgs'))])

scores = cross_val_score(estimator
                         , X, y
                         , scoring='roc_auc', cv=5)


print('AUROC for all folds:')
print(scores)
print('Average AUROC across folds:')
print(scores.mean())

AUROC for all folds:
[0.61605425 0.64243601 0.62164081 0.60277423 0.61229702]
Average AUROC across folds:
0.6190404642649556


In [14]:
# train pipeline
estimator.fit(X_train, y_train)

# predict class labels for the test set
y_pred = model.predict(X_test)

# generate class probabilities
y_prob = model.predict_proba(X_test)

# generate evaluation metrics
print('Accuracy = {}'.format(accuracy_score(y_test, y_pred)))
print('AUROC = {}'.format(roc_auc_score(y_test, y_prob[:, 1])))

Accuracy = 0.8951816039652231
AUROC = 0.6033460168207213


In [15]:
# test saving and loading model locally
from sklearn.externals import joblib
import os

//anaconda3/envs/sagemaker-local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
joblib.dump(estimator, os.path.join('./', "model.joblib"))

['./model.joblib']

In [17]:
estimator = joblib.load(os.path.join('./', "model.joblib"))

estimator

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regression',
                 LogisticRegressionCV(Cs=10, class_weight=None, cv=5,
                                      dual=False, fit_intercept=True,
                                      intercept_scaling=1.0, l1_ratios=None,
                                      max_iter=100, multi_class='warn',
                                      n_jobs=None, penalty='l2',
                                      random_state=None, refit=True,
                                      scoring='roc_auc', solver='lbfgs',
                                      tol=0.0001, verbose=0))],
         verbose=False)

## 6. Deployment
We finally deploy in SageMaker, in a managed training and hosting infrastructure:

For more information about SageMaker, get out the [docs](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html).

Remember to add permissions of SageMakerFullAccess to IAM Role of the notebook instance.

In [18]:
# S3 prefix
prefix = 'Scikit-mimic'

import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import numpy as np
import os

sagemaker_session = sagemaker.Session()

try:
    # Get a SageMaker-compatible role used by this Notebook Instance (using notebook instance).
    role = sagemaker.get_execution_role()
except ValueError:
    # Get a SageMaker-compatible role pre-configured for the Notebook Instance (using SageMaker locally).
    print('Using notebook locally and pre-configured IAM role in the AWS cloud...')
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='MIMIC-ExecutionRole-IXCZDX872KXW')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name mccunha to get Role path.


Using notebook locally and pre-configured IAM role in the AWS cloud...


In this example there's already a pre-configured role called "MIMIC-ExecutionRole-IXCZDX872KXW". 

Also, my AWS credentials configured locally in my laptop and associated to my user give me permissions to perform [GetRole](https://docs.aws.amazon.com/IAM/latest/APIReference/API_GetRole.html) from IAM and [PutObject](https://docs.aws.amazon.com/AmazonS3/latest/API/API_PutObject.html) in S3.

In [19]:
# Create directory and write csv
os.makedirs('./data', exist_ok=True)
np.savetxt('./data/mimic.csv', data, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')

In [20]:
# Save data in S3 for training with SageMaker
data_dir = 'data'
train_input = sagemaker_session.upload_data(data_dir, key_prefix="{}/{}".format(prefix, data_dir) )

Before submiting to SageMaker in the cloud instances, let's test our code with the Scikit-Learn container locally (using `train_instance_type="local"`):

Obs.: Since your local machine will need to pull the docker image, **remember to ensure that Docker is running**.

In [21]:
script_path = 'scikit_learn_mimic.py'

# Simple training without hyperparameters
# The entry script in a python script based on the code developed above
sklearn_local = SKLearn(
    entry_point=script_path,
    framework_version='0.23-1',
    train_instance_type="local",
    role=role,
    hyperparameters={},
    metric_definitions=[
            {'Name':'test:accuracy', 'Regex':'Test Accuracy: (.*?);'},
            {'Name':'test:auroc', 'Regex':'Test AUROC: (.*?);'}
    ]
)

When running locally, it will take a few minutes for the first run, since under the hood SageMaker SDK will pull the [Scikit-Learn Docker image from the ECR repository](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-frameworks.html) and image has a over 1GB.

In [22]:
sklearn_local.fit({'train': train_input})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Creating tmpp0mdzc25_algo-1-25n6g_1 ... 
Attaching to tmpp0mdzc25_algo-1-25n6g_12mdone
algo-1-25n6g_1  | 2020-07-23 00:25:25,998 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
algo-1-25n6g_1  | 2020-07-23 00:25:26,010 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-25n6g_1  | 2020-07-23 00:25:26,046 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-25n6g_1  | 2020-07-23 00:25:27,125 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-25n6g_1  | 2020-07-23 00:25:27,175 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-25n6g_1  | 2020-07-23 00:25:27,223 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-25n6g_1  | 2020-07-23 00:25:27,271 sagemaker-training-toolkit INFO     Invoking user script
algo-1-25n6g_1  | 
algo-1-25n6g_1  | Training Env:
algo-1-25n

---
If you'd like to test locally, pull the image in the terminal:
![docker](./media/scikit-docker-image.png)

Under the hood the SageMaker local mode uses Docker compose to setup the local trainig. You can run the it in your terminal for troubleshooting:

```
!docker-compose -f /private/var/folders/s8/qvzyzg3d4p1241_h8yt0l92152zgzv/T/tmpj8a2pib4/docker-compose.yaml up --build --abort-on-container-exit
```

---
Everything seems to be working fine. Now, let's train in the managed SageMaker instances:

In [23]:
script_path = 'scikit_learn_mimic.py'

# Simple training without hyperparameters
# The entry script in a python script based on the code developed above
sklearn = SKLearn(
    framework_version='0.23-1',
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={},
    metric_definitions=[
            {'Name':'test:accuracy', 'Regex':'Test Accuracy: (.*?);'},
            {'Name':'test:auroc', 'Regex':'Test AUROC: (.*?);'}
    ]
)

In [24]:
# We want to see and stream logs from our container training in AWS with SageMaker. Therefore we leave `wait=True`
sklearn.fit({'train': train_input}, wait=True)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-07-23 00:25:34 Starting - Starting the training job...
2020-07-23 00:25:38 Starting - Launching requested ML instances.........
2020-07-23 00:27:20 Starting - Preparing the instances for training...
2020-07-23 00:28:15 Downloading - Downloading input data...
2020-07-23 00:28:39 Training - Downloading the training image...
2020-07-23 00:29:22 Training - Training image download completed. Training in progress..2020-07-23 00:29:22,239 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-07-23 00:29:22,240 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-23 00:29:22,250 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-23 00:29:22,600 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-23 00:29:24,017 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-23 00:29:24,028 sagemaker-training-to

You should see SageMaker spinning up instances for training:

![sm-train](./media/sagemaker-training.png)

---
Again, we can test the deploy locally before deploying in the SageMaker hosting services:

In [25]:
predictor_local = sklearn_local.deploy(initial_instance_count=1, instance_type="local")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmpkxislc6d_algo-1-sqsdt_1
algo-1-sqsdt_1  | 2020-07-23 00:30:17,487 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-sqsdt_1  | 2020-07-23 00:30:17,493 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-sqsdt_1  | 2020-07-23 00:30:17,494 INFO - sagemaker-containers - nginx config: 
algo-1-sqsdt_1  | worker_processes auto;
algo-1-sqsdt_1  | daemon off;
algo-1-sqsdt_1  | pid /tmp/nginx.pid;
algo-1-sqsdt_1  | error_log  /dev/stderr;
algo-1-sqsdt_1  | 
algo-1-sqsdt_1  | worker_rlimit_nofile 4096;
algo-1-sqsdt_1  | 
algo-1-sqsdt_1  | events {
algo-1-sqsdt_1  |   worker_connections 2048;
algo-1-sqsdt_1  | }
algo-1-sqsdt_1  | 
algo-1-sqsdt_1  | http {
algo-1-sqsdt_1  |   include /etc/nginx/mime.types;
algo-1-sqsdt_1  |   default_type application/octet-stream;
algo-1-sqsdt_1  |   access_log /dev/stdout combined;
algo-1-sqsdt_1  | 
algo-1-sqsdt_1  |   upstream gunicorn {
algo-1-sqsdt_1  |     server unix:/tmp/guni

In [26]:
# predict class labels for the test set
y_pred_endpoint = predictor_local.predict(X_test)
print('shape of predictions array:',y_pred_endpoint.shape)

# generate evaluation metrics
print('Accuracy = {}'.format(accuracy_score(y_test, y_pred)))
print('AUROC = {}'.format(roc_auc_score(y_test, y_prob[:, 1])))

print('\nConfusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\nClassification report')
print(classification_report(y_test, y_pred))

algo-1-sqsdt_1  | 2020-07-23 00:30:25,778 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
shape of predictions array:algo-1-sqsdt_1  | 172.18.0.1 - - [23/Jul/2020:00:30:26 +0000] "POST /invocations HTTP/1.1" 200 98584 "-" "-"
 (12307,)
Accuracy = 0.8951816039652231
AUROC = 0.6033460168207213

Confusion matrix
[[10964    15]
 [ 1275    53]]

Classification report
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     10979
           1       0.78      0.04      0.08      1328

    accuracy                           0.90     12307
   macro avg       0.84      0.52      0.51     12307
weighted avg       0.88      0.90      0.85     12307



In [27]:
predictor_local.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


Everything seems to be working fine. Now, deploy in the AWS cloud:

In [28]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

You should see SageMaker spinning up instances for hosting the model with an API endpoint:

![sm-train](./media/sagemaker-hosting.png)

![sm-train](./media/sagemaker-hosting-2.png)

In [29]:
# predict class labels for the test set
y_pred_endpoint = predictor.predict(X_test)

# generate evaluation metrics
print('Accuracy = {}'.format(accuracy_score(y_test, y_pred)))
print('AUROC = {}'.format(roc_auc_score(y_test, y_prob[:, 1])))

print('\nConfusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\nClassification report')
print(classification_report(y_test, y_pred))

Accuracy = 0.8951816039652231
AUROC = 0.6033460168207213

Confusion matrix
[[10964    15]
 [ 1275    53]]

Classification report
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     10979
           1       0.78      0.04      0.08      1328

    accuracy                           0.90     12307
   macro avg       0.84      0.52      0.51     12307
weighted avg       0.88      0.90      0.85     12307



In [30]:
# Delete endpoint for avoid costs
sklearn.delete_endpoint()